In [8]:
def convert_line_old(line, lineno=None):
    parts = line.split(':    ')
    return '\t'.join(parts)

In [20]:
def convert_line(line):
    ix = line.find(b':')
    assert ix != -1
    offset = line[:ix]
    rest = line[ix+5:]
    return b'\t'.join((offset, rest))

In [21]:
def convert_file(path):
    with open(path, 'rb') as src:
        with open(path.replace('.txt', '.tsv'), 'wb') as dst:
            for (srclineno, srcline) in enumerate(src):
                try:
                    dst.write(convert_line(srcline, srclineno) + '\r')
                except:
                    print("[%d]: %s" % (srclineno, srcline))
                

In [22]:
line = b'2615:    AccessibleComputing'

In [23]:
convert_line(line)

b'2615\tAccessibleComputing'

In [1]:
import string
import datrie

In [2]:
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [3]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [4]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
allowed = (
    string.printable +
    string.punctuation
)

In [33]:
titles_trie = datrie.Trie(allowed)
titles_offset_trie = datrie.Trie(string.digits)
titles_lower_trie = datrie.Trie(string.ascii_lowercase + string.punctuation)

In [34]:
def convert_word(line):
    ix = line.find('\t')
    assert ix != -1
    offset = line[:ix]
    if line[-1] == '\n':
        word = line[ix+1:-1]
    else:
        word = line[ix+1:]
    return (word, offset)

In [35]:
def add_to_trie(trie, offset_trie, key, value):
    offset = int(value)
    if key in trie:
        existing = trie[key]
        if value not in existing:
            existing.append(offset)
            existing.sort()
    else:
        trie[key] = [ offset, ]
    
    if value in offset_trie:
        existing = offset_trie[value]
        if key not in existing:
            existing.append(key)
            existing.sort()
    else:
        offset_trie[value] = [ key, ]
    
    lower_value = key.lower()
    if lower_value == value:
        return

    if lower_value not in trie:
        trie[lower_value] = [ -offset, ]

In [36]:
def add_word(line):
    (word, offset) = convert_word(line)
    add_to_trie(titles_trie, titles_offset_trie, word, offset)

In [37]:
failed = []

In [38]:
max_failed = 10
with open('titles.tsv', 'rb') as f:
    for bytes_line in f:
        try:
            line = bytes_line.decode('utf-8', 'replace')
            add_word(line)
        except Exception as e:
            failed.append((e, line))
            if len(failed) >= max_failed:
                raise

In [39]:
titles_trie.save('titles.trie')

In [40]:
titles_offset_trie.save('titles_by_offset.trie')

In [18]:
failed = None

In [41]:
len(failed)

0

In [42]:
len(titles_trie)

27962169

In [ ]:
!conda install numpy

In [6]:
import numpy as np

In [7]:
titles_offset = datrie.Trie.load('titles_by_offset.trie')

In [10]:
titles_trie = datrie.Trie.load('titles.trie')

In [11]:
len(titles_offset)

15271374

In [9]:
%timeit 'Python' in titles_offset

The slowest run took 98.87 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 80.4 ns per loop


In [39]:
len(titles_trie.prefixes(' (disambiguation)'))

0

In [54]:
titles_trie.items('Python')

[('Python', [33919833364]),
 ('Python (Asterophis) tigris', [20455085613]),
 ('Python (Busch Gardens Tampa Bay)', [14147158870]),
 ('Python (Coney Island)', [49590305361]),
 ('Python (Coney Island, Cincinnati, Ohio)', [26110321609]),
 ('Python (Coney island)', [26110331669]),
 ('Python (Coney island,Cincinnati, Ohio)', [26110350476]),
 ('Python (Efteling)', [18631684627]),
 ('Python (Geographic range)', [26538791922]),
 ('Python (Greek mythology)', [10394169050]),
 ('Python (Monty)', [8734521746]),
 ('Python (Monty) Pictures', [20062064793]),
 ('Python (Programming Language)', [33053541377]),
 ('Python (automobile maker)', [9559978406]),
 ('Python (computer language)', [26034782009]),
 ('Python (computing)', [41806032476]),
 ('Python (disambiguation)', [33919839039]),
 ('Python (film)', [4228420425]),
 ('Python (genus)', [12243555802]),
 ('Python (lang)', [21890868716]),
 ('Python (language)', [4151314363]),
 ('Python (missile)', [2808088550]),
 ('Python (movie)', [8565470739]),
 ('Pyt

In [14]:
offsets = titles_offset.items('339198')

In [68]:
offsets_list = [ np.uint64(a[0]) for a in titles_offset.items() ]

In [26]:
a = np.array(offsets, dtype=np.uint64)

In [40]:
len(a)

15271374

In [29]:
np.save('titles_offsets.npy', a)

In [35]:
%timeit np.searchsorted(a, np.uint64(33919833364), side='left')

The slowest run took 26.55 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.17 µs per loop


In [42]:
np.searchsorted(a, np.uint64(33919833364), side='left')

7763814

In [32]:
np.searchsorted(a, np.uint64(33919833364), side='right')

7763815

In [34]:
a[7763813]

33919832665

In [25]:
len(offsets)

15271374

In [21]:
a = np.array(offsets, dtype=np.uint64)

In [23]:
b = np.array((a[0] for a in titles_offset.items()), dtype=np.uint64)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'generator'

In [15]:
len(offsets)

47

In [29]:
failed[0]

(TypeError('unorderable types: int() < tuple()'),
 '886354688\t1911 encyclopedia\n')

In [45]:
words.save('words.trie')

In [46]:
offset_to_words.save('words_by_offset.trie')

In [47]:
%timeit datrie.Trie.load('words.trie')

1 loops, best of 3: 1.95 s per loop


In [48]:
words.keys()[:10]

['A',
 'Aani',
 'Aaron',
 'Aaronic',
 'Aaronical',
 'Aaronite',
 'Aaronitic',
 'Aaru',
 'Ab',
 'Ababdeh']

In [35]:
%timeit words.prefixes('aaron')

The slowest run took 18.40 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 195 ns per loop


In [44]:
%timeit words['aaron']

The slowest run took 21.18 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 109 ns per loop


In [47]:
words.suffixes('Aa')

['ron']

In [49]:
words.items('aa')

[('aaron', [44])]

In [55]:
def get_enclosing_offsets(o, offsets):
    ix = np.searchsorted(offsets, np.uint64(o), side='left')
    return (offsets[ix-1], offsets[ix+1])
    

In [18]:
def get_enclosing_offsets2(o, offsets):
    ix = offsets.searchsorted(np.uint64(o), side='right')
    return (o-7, int(offsets[ix]-np.uint64(8)))

In [11]:
import numpy as np

In [15]:
!cd

d:\data


In [16]:
offsets = np.load('titles_offsets3.npy', mmap_mode=None)

In [17]:
offsets.dtype

dtype('uint64')

In [45]:
titles = titles_trie

In [47]:
%timeit titles['Python']

The slowest run took 57.90 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 129 ns per loop


In [48]:
titles['Python']

[33919833364]

In [49]:
offsets.sort()

In [50]:
sorted_offsets = np.sort(offsets)

In [51]:
len(sorted_offsets)

15271374

In [52]:
old_offsets = offsets

In [53]:
offsets = sorted_offsets

In [54]:
np.save('titles_offsets.npy', offsets)

In [59]:
x = get_enclosing_offsets(titles['Python'][0], offsets)

In [61]:
type(x[0])

numpy.str_

In [66]:
%timeit offsets.searchsorted(np.uint64(33919833364))

1 loops, best of 3: 473 ms per loop


In [67]:
type(old_offsets)

list

In [69]:
offsets = None

In [1]:
txt = """  <page>
    <title>Python</title>"""

In [6]:
txt.find('<')

AttributeError: 'str' object has no attribute 'findall'

In [3]:
titles['Python']

NameError: name 'titles' is not defined

In [4]:
33919833364-20

33919833344

In [7]:
33919833364

33919833364

In [20]:
uint64_8 = np.uint64(8)

In [43]:
uint64_10 = uint64(10)
uint64_11 = uint64(11)

In [48]:
from numpy import uint64
uint64_7 = np.uint64(7)
uint64_11 = uint64(11)
def get_enclosing_offsets3(o):
    ix = offsets.searchsorted(o, side='right')
    return (int(o-uint64_7), int(offsets[ix]-uint64_11))

In [50]:
from numpy import uint64
uint64_7 = np.uint64(7)
uint64_11 = uint64(11)
def get_enclosing_offsets4(o):
    ix = offsets.searchsorted(o, side='right')
    return (o-uint64_7, offsets[ix]-uint64_11)

In [52]:
%timeit get_enclosing_offsets4(uint64(4388827238))

The slowest run took 11.40 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.76 µs per loop


In [46]:
def offsets_to_range(pair):
    return '-'.join([str(pair[0]), str(pair[1])])

In [54]:
%timeit offsets_to_range(get_enclosing_offsets4(uint64(4388827238)))

The slowest run took 14.67 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 6.56 µs per loop


In [55]:
offsets_to_range(get_enclosing_offsets4(uint64(4388827238)))

'4388827231-4388830599'

In [56]:
import datrie

In [57]:
small_trie = datrie.Trie.load('titles-1m.trie')

In [59]:
small_trie.keys()[100:200]

['.Mac',
 '.Net',
 '.SPC',
 '.TV',
 '.am',
 '.au (domain name)',
 '.cd',
 '.com',
 '.eu',
 '.fm',
 '.hack (games)',
 '.hack (video game series)',
 '.hack//Sign',
 '.hack//sign',
 '.hack/sign',
 '.hk',
 '.mac',
 '.moe',
 '.mp3',
 '.net',
 '.nl',
 '.org',
 '.spc',
 '.tv',
 '/',
 '/.ed',
 '0 (number)',
 '0 Series Shinkansen',
 '0 series shinkansen',
 '0-9',
 '0.1km2',
 '0/1 knapsack problem',
 '007 (disambiguation)',
 '00s',
 '0X',
 '0s',
 '0s BC',
 '0s bc',
 '0x',
 '1',
 '1 (Beatles album)',
 '1 (New York City Subway service)',
 '1 (beatles album)',
 '1 (new york city subway service)',
 '1 (number)',
 '1 (the number)',
 '1 April',
 '1 August',
 '1 BC',
 '1 Chronicles',
 '1 Corinthians',
 '1 December',
 '1 E -1 kg',
 '1 E -1 m',
 '1 E -10 kg',
 '1 E -10 m',
 '1 E -11 kg',
 '1 E -11 m',
 '1 E -12 kg',
 '1 E -13 J',
 '1 E -13 kg',
 '1 E -13 m',
 '1 E -14 J',
 '1 E -14 kg',
 '1 E -15 kg',
 '1 E -15 m',
 '1 E -16 kg',
 '1 E -16 s',
 '1 E -17 kg',
 '1 E -18 kg',
 '1 E -19 kg',
 '1 E -2 kg',
 '

In [62]:
small_trie.keys('.hack')

['.hack (games)',
 '.hack (video game series)',
 '.hack//Sign',
 '.hack//sign',
 '.hack/sign']

In [69]:
small_trie.items('Alph')

[('Alph', [37796646]),
 ('Alpha', [9485835, 9485835, 9485835]),
 ('Alpha (disambiguation)', [1906473862]),
 ('Alpha 1-antitrypsin deficiency', [2767914001]),
 ('Alpha Arietis', [1110148856]),
 ('Alpha Blondy', [1123384012]),
 ('Alpha Centauri', [27211921]),
 ('Alpha Centauri A', [2190432187]),
 ('Alpha Centauri B', [2190432762]),
 ('Alpha Centauri computer game', [2315351167]),
 ('Alpha Centauri/Videogame', [1162750267]),
 ('Alpha Crucis', [1928475714]),
 ('Alpha Herculis', [1197049692]),
 ('Alpha centauri', [1889461987]),
 ('Alpha channel', [2286099071]),
 ('Alpha compositing', [28622449]),
 ('Alpha decay', [15317609, 15317609, 15317609]),
 ('Alpha emission', [898135668]),
 ('Alpha helix', [43805509]),
 ('Alpha process', [2268016863]),
 ('Alpha processor', [931824748]),
 ('Alpha radiation', [2080753887]),
 ('Alpha ray', [15039418, 15039418, 15039418]),
 ('Alpha receptor', [2884222826]),
 ('Alpha version', [2286985487]),
 ('Alpha, Illinois', [1363133034]),
 ('Alpha, Michigan', [1437482

In [78]:
%timeit small_trie.items('Al')

100 loops, best of 3: 4.16 ms per loop


In [74]:
len(small_trie.items('Al'))

2295

In [76]:
from collections import defaultdict

In [ ]:
from numpy import uint64
uint64_7 = np.uint64(7)
uint64_11 = uint64(11)
def get_enclosing_offsets4(o):
    ix = offsets.searchsorted(o, side='right')
    return (o-uint64_7, offsets[ix]-uint64_11)

In [85]:
from numpy import uint64
uint64_7 = np.uint64(7)
uint64_11 = uint64(11)
def get_page_offsets_for_key(search, trie, offsets):
    items = trie.items(search)
    results = defaultdict(list)
    for (key, value) in items:
        for v in value:
            o = uint64(v if v > 0 else v*-1)
            ix = offsets.searchsorted(o, side='right')
            results[key].append((int(o-uint64_7), int(offsets[ix]-uint64_11)))
    return results or None

In [113]:
from numpy import uint64
uint64_7 = np.uint64(7)
uint64_11 = uint64(11)
def get_page_offsets_for_key2(search, trie, offsets):
    items = trie.items(search)
    results = []
    for (key, value) in items:
        for v in value:
            o = uint64(v if v > 0 else v*-1)
            ix = offsets.searchsorted(o, side='right')
            results.append((key, (int(o-uint64_7), int(offsets[ix]-uint64_11))))
    return results

In [ ]:
from numpy import uint64
uint64_7 = np.uint64(7)
uint64_11 = uint64(11)
def get_page_offsets_for_key3(search, trie, offsets, offset_trie):
    items = trie.items(search)
    results = []
    for (key, value) in items:
        for v in value:
            if v < 0:
                
            o = uint64(v if v > 0 else v*-1)
            ix = offsets.searchsorted(o, side='right')
            results.append((key, (int(o-uint64_7), int(offsets[ix]-uint64_11))))
    return results

In [115]:
%timeit get_page_offsets_for_key2('alpha', small_trie, offsets)

1000 loops, best of 3: 205 µs per loop


In [90]:
get_page_offsets_for_key2('Alphonso', small_trie, offsets)

[('Alphonso', (1869195953, 1869196572)),
 ('Alphonso D Albuquerque', (849735505, 849736079)),
 ('Alphonso I', (1869354079, 1869354680)),
 ('Alphonso I of Portugal', (2750663170, 2750663805)),
 ('Alphonso I of Spain', (21890018, 21891191)),
 ('Alphonso I of Spain', (21890018, 21891191)),
 ('Alphonso I of Spain', (21890018, 21891191)),
 ('Alphonso II', (1869353487, 1869354075)),
 ('Alphonso II of Aragon', (1869209074, 1869209708)),
 ('Alphonso II of Asturies', (1869182884, 1869183524)),
 ('Alphonso II of Portugal', (2750663809, 2750664431)),
 ('Alphonso II of Spain', (1735538773, 1735539348)),
 ('Alphonso III', (1869352876, 1869353483)),
 ('Alphonso III of Aragon', (1869304626, 1869305247)),
 ('Alphonso III of Leon', (1869183528, 1869184155)),
 ('Alphonso III of Portugal', (2750665687, 2750666328)),
 ('Alphonso IV', (1869354684, 1869355272)),
 ('Alphonso IV of Aragon', (1869305251, 1869305885)),
 ('Alphonso IV of Leon', (1869192685, 1869193302)),
 ('Alphonso IV of Portugal', (2750667635,

In [91]:
small_trie.items('Alphonso')

[('Alphonso', [1869195960]),
 ('Alphonso D Albuquerque', [849735512]),
 ('Alphonso I', [1869354086]),
 ('Alphonso I of Portugal', [2750663177]),
 ('Alphonso I of Spain', [21890025, 21890025, 21890025]),
 ('Alphonso II', [1869353494]),
 ('Alphonso II of Aragon', [1869209081]),
 ('Alphonso II of Asturies', [1869182891]),
 ('Alphonso II of Portugal', [2750663816]),
 ('Alphonso II of Spain', [1735538780]),
 ('Alphonso III', [1869352883]),
 ('Alphonso III of Aragon', [1869304633]),
 ('Alphonso III of Leon', [1869183535]),
 ('Alphonso III of Portugal', [2750665694]),
 ('Alphonso IV', [1869354691]),
 ('Alphonso IV of Aragon', [1869305258]),
 ('Alphonso IV of Leon', [1869192692]),
 ('Alphonso IV of Portugal', [2750667642]),
 ('Alphonso IX', [22224716, 22224716, 22224716]),
 ('Alphonso IX of Castile', [1869351046]),
 ('Alphonso IX of Spain', [1869195223]),
 ('Alphonso Raymond of Castile', [2780947101]),
 ('Alphonso V', [1869355283]),
 ('Alphonso V of Aragon', [853469055]),
 ('Alphonso V of Cast

In [110]:
import time

In [93]:
import sys
err = sys.stderr.write

In [111]:
class timer:
    def __init__(self, verbose=False):
        self.start = None
        self.stop = None
        self.elapsed = None
        self.nsec = None
        self.msec = None
        self.mill = None
        self.fmt = None
        self.verbose = verbose
        
    def __str__(self):
        return self.fmt
    
    def __repr__(self):
        return self.fmt
        
    def __enter__(self):
        self.start = time.clock()
        
    def __exit__(self, *exc_info):
        self.stop = time.clock()
        self.elapsed = self.stop - self.start
        self.nsec = self.elapsed * 1e9
        self.msec = self.elapsed * 1e6
        self.mill = self.elapsed * 1e3
        if self.nsec < 1000:
            self.fmt = "%dns" % self.nsec
        elif self.msec < 1000:
            self.fmt = "%dµs" % self.msec
        elif self.mill < 1000:
            self.fmt = "%dms" % self.mill
        else:
            self.fmt = "%0.3fs" % self.elapsed
        
        if self.verbose:
            print(self.fmt)
    
    @classmethod
    def timeit(cls):
        return cls(verbose=True)

In [112]:
with timer.timeit():
    small_trie.items('Alphonso')

128µs


In [99]:
with timer():
    pass

1.539556251373142e-06


In [100]:
r = 1.539556251373142e-06


In [102]:
r < 1e-6

False

In [103]:
r

1.539556251373142e-06

In [104]:
r * 1e6

1.539556251373142

In [116]:
with timer.timeit():
    dummy2 = get_page_offsets_for_key2('alphonso', titles, offsets)

3µs


NameError: name 'titles' is not defined

In [122]:
state = datrie.State(small_trie)

In [126]:
state.walk('alphonso')

False

In [127]:
it = datrie.Iterator(state)

In [128]:
[ (it.key(), it.data()) for i in range(0, 9) if it.next() ]

[('', [1869195960]),
 (' D Albuquerque', [849735512]),
 (' I', [1869354086]),
 (' I of Portugal', [2750663177]),
 (' I of Spain', [21890025, 21890025, 21890025]),
 (' II', [1869353494]),
 (' II of Aragon', [1869209081]),
 (' II of Asturies', [1869182891]),
 (' II of Portugal', [2750663816])]

In [129]:
state??


In [130]:
dir(state)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 'copy_to',
 'data',
 'is_leaf',
 'is_single',
 'is_terminal',
 'rewind',
 'walk']

In [131]:
datrie.State.